## Ceneo Scraper

## Struktura opinii w serwisie ceneo.pl

|składowa|Selektor|zmienna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autora|.user-post__author-name|author|
|rekomendację|.user-post__author-recomendation >  em|recommend|
|liczbę gwiazdek|.user-post__score-count|stars|
|data wystawienia opinii|span.user-post__published > time::nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time::nth-child(2)["datetime"]|purchase_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|
|treść opinii|.user-post__text|content|
|listę wad|div.review-feature__title--negatives~div.review-feature__item|cons|
|listę zalet|div.review-feature__title--positives~div.review-feature__col|pros|

1. Import Bibliotek

In [53]:
selectors = {
'opinion_id': (None, 'data-entry-id'),
'author': ('span.user-post__author-name', ),
'recommend': ('span.user-post__author-recomendation > em',),
'stars': ('span.user-post__score-count',),
'content': ('div.user-post__text',),
'pros': ('div.review-feature__title--negatives ~ div.review-feature__item', None, True),
'cons': ('div.review-feature__title--positives ~ div.review-feature__item', None, True),
'opinion_date': ('span.user-post__published > time:nth-child(1)', "datetime"),
'purchase_date': ('span.user-post__published > time:nth-child(2)', "datetime"),
'up_vote': ('button.vote-yes', 'data-total-vote'),
'down_vote': ('button.vote-no', 'data-total-vote')
}

In [54]:
import json
import os
import requests
from bs4 import BeautifulSoup

2. Wczytanie kodu produktu  o którym mają zostać wczytane opinie


In [55]:
# product_id ='17469616'
product_id = input('Podaj kod produktu: ')
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

3. Wysłanie do serwera Ceneo.pl żądania dostępu do strony z opiniami o produkcie

In [56]:
response=requests.get(url)
response.status_code

200

4. Przekształcenie kodu html w postaci tekstowej do obiektu reprezentującego strukturę DOM
5. Wydobycie ze struktry DOm znacznikow odpowiadających opiniom konsumentów

In [57]:
page = BeautifulSoup(response.text,"html.parser")
opinions=page.select("div.js_product-review")

3.Pobranie wszystkich opinii o produkcie z serwisu ceneo.pl

In [59]:
all_opinions =[]
while(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    opinions = page.select('div.js_product-review')
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + extract(page, "a.pagination__next", 'href')
    except  TypeError:
        url = None

NameError: name 'extract' is not defined

6. Wydobycie ze struktury znacznika odpowiadającego pojedynczej opinii jej składowych

In [ ]:
def extract(ancestor, selector = None, attribute = None, return_list= False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip

In [ ]:
#single_opinion

4. Zapis opinii o produkcie do pliku JSON do katalogu opinions

In [ ]:
if not os.path.exists('opinions'):
    os.mkdir('opinions')
with open(f'opinions/{product_id}.json', 'w', encoding='UTF-8') as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

In [ ]:
all_opinions

[{'opinion_id': '3390567',
  'author': 'wlad-c',
  'recommend': 'Polecam',
  'stars': '5/5',
  'content': 'Duża  moc,spokojnie zastępuje mniejsze kolby.',
  'pros': [],
  'cons': ['moc'],
  'opinion_date': '2015-10-29 14:50:59',
  'purchase_date': '2015-10-08 21:20:56',
  'up_vote': '4',
  'down_vote': '0'},
 {'opinion_id': '17054967',
  'author': 'Piotr',
  'recommend': 'Polecam',
  'stars': '5/5',
  'content': 'Lekka, mocna (Błyskawicznie się nagrzewa). Wymienne końcówi są bardzo przydatne. No i na koniec atrakcyjna cena.',
  'pros': [],
  'cons': [],
  'opinion_date': '2023-01-24 14:18:03',
  'purchase_date': '2023-01-10 19:11:04',
  'up_vote': '1',
  'down_vote': '0'},
 {'opinion_id': '6388218',
  'author': 'Natalia',
  'recommend': 'Polecam',
  'stars': '5/5',
  'content': 'produkt jest w porządku.  zgrabna walizeczka dzięki której nic się nie gubi. polecam',
  'pros': [],
  'cons': [],
  'opinion_date': '2017-12-23 12:46:51',
  'purchase_date': '2017-12-05 12:38:27',
  'up_vote':